In [ ]:
from Camera import Camera
import numpy as np
from Frame import Frame
from plyfile import PlyData
import numpy as np
import argparse
from io import BytesIO
import matplotlib.pyplot as plt
import Plotters
import plotly
import plotly.graph_objects as go
import scipy.io
import pandas as pd
import Utils
from GaussianSplat import GaussianSplat
import Plotters
from Render import Render

%matplotlib qt

# load hull
path = 'G:/My Drive/Research/gs_data/mov19_2022_03_03/'
real_coord = scipy.io.loadmat(f'{path}/3d_pts/real_coord.mat')['all_coords']
points_3d = {body_wing : pd.DataFrame(Utils.load_hull(body_wing,path),columns = ['X','Y','Z','frame']) for body_wing in ['body','rwing','lwing']}
# initilize objects
frames = [1407]#list(range(500,700,13))

image_name,points_in_idx = Utils.define_frames(frames,points_3d)
cameras = {f'cam{cam + 1}':Camera(path,cam) for cam in range(4)}
frames = {f'{im_name}.jpg':Frame(path,im_name,points_in_idx[im_name.split('CAM')[0]],real_coord,idx) for idx,im_name in enumerate(image_name)}
# map 3d voxels to 2d pixels
[frames[im_name].map_3d_2d(croped_image = True) for im_name in frames.keys()]
voxel_dict,colors_dict = Utils.get_dict_for_points3d(frames)


In [ ]:
input_file = "G:/My Drive/Research/gs_data/mov19_2022_03_03/output/scaling_lr0.005_percent_dense0.001_densify_grad_threshold0.0002_position_lr_init1.6e-07/point_cloud/iteration_30000/point_cloud.ply"
gs = GaussianSplat(input_file)  

gs_filtered = gs.filter(gs.projection_filter(frames,gs.xyz,croped_image = True),path = gs.path)
gs_filtered.save_gs()



In [ ]:

idx_to_clamp = (gs_filtered.color[:,0] >= 0) & (gs_filtered.color[:,1] >= 0) & (gs_filtered.color[:,2] >= 0) & (gs_filtered.color[:,0] <=1) & (gs_filtered.color[:,1] <=1) & (gs_filtered.color[:,2] <=1)
sorted_gs = GaussianSplat(vertices = gs_filtered.vertices[idx_to_clamp], sh = gs_filtered.sh[idx_to_clamp])



cam = frames[image_name].get_croped_camera()
render = Render(gs,cam,tiles = [1,10],block_xy = [16,16], image_size = [160,160])
image = render.render_image()
plt.figure(),plt.imshow(image)




Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


(<Figure size 640x480 with 1 Axes>,
 <matplotlib.image.AxesImage at 0x1c283c0ef80>)